In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import transformers

In [3]:
model_class, tokenizer_class, name = (transformers.BertModel, transformers.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(name)
model = model_class.from_pretrained(name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
dir = '/content/gdrive/My Drive/Colab Notebooks/cs4248/Project/'
train_file = dir + "fulltrain.csv"
test_file = dir + "balancedtest.csv"

In [6]:
column_name = ['label', 'text']
train_df = pd.read_csv(train_file, names=column_name)
test_df = pd.read_csv(test_file, names=column_name)

In [7]:
train_text = train_df["text"].values
test_text = test_df['text'].values

In [8]:
def encode(text):
  return tokenizer.encode_plus(text, add_special_tokens = True, truncation = True, padding = "max_length", return_attention_mask = True, return_tensors = "pt", max_length=200)

In [11]:
train_text = [encode(text) for text in train_text]

In [ ]:
test_text = [encode(text) for text in test_text]

In [12]:
train_ids = [encoding['input_ids'] for encoding in train_text]
train_attention_mask = [encoding["attention_mask"] for encoding in train_text]

In [ ]:
test_ids = [encoding['input_ids'] for encoding in test_text]
test_attention_mask = [encoding["attention_mask"] for encoding in test_text]

In [13]:
from datetime import datetime
import torch

In [19]:
train_output = []

for i in range(5):
  if (i % 100 == 0):
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S") # in UTC for reference
    print("Current Time =", current_time)
    print("running the " + str(i) + "-th loop")

  ids = train_ids[i]
  mask = train_attention_mask[i]

  with torch.no_grad():
    hidden_reps  = model(ids, attention_mask = mask)

  train_output.append(np.array(hidden_reps[0][:,0,:][0]))  

Current Time = 11:29:22
running the 0-th loop


In [20]:
train_output = np.array(train_output)

In [21]:
train_output.shape

(5, 768)

In [22]:
new_df = pd.DataFrame(train_output)
new_df['label'] = train_df['label'].values[:5]

In [24]:
new_df.to_csv(dir + "newtrain.csv", index=False)

In [ ]:
train_output = np.array(train_output)

In [ ]:
new_train_df = pd.DataFrame(train_output)
new_train_df['label'] = train_df['label']

In [ ]:
new_train_df.head()

In [ ]:
train_text = np.array(train_df['text'])
test_text = np.array(test_df['text'])